In [1]:
import sys
# !{sys.executable} -m pip install shap
from glob import glob
import numpy as np
import joblib
import os
import pandas as pd
import json

import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from tqdm.notebook import tqdm
import seaborn as sns
from collections import Counter

# SHAP
import shap

In [2]:
# glob(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'database','*.csv.gz'))

In [3]:
trainsize=[ 0.1,0.3, 0.5, 0.7,0.8]

In [4]:


gaussian_kernel = RBF()

# remove basic linear models (ridge, lasso, elastic) and expensive models (gpr)

models = {
    # 'ridge': Ridge(),
    # 'lasso': Lasso(),
    # 'elastic': ElasticNet(),
    'knn': KNeighborsRegressor(),
    'rfr': RandomForestRegressor(),
    # 'grad': GradientBoostingRegressor(),
    # 'svr': SVR(),
    'krr': KernelRidge(),
    # 'gpr': GaussianProcessRegressor()
}

param_grid = {
    'ridge': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 50, 100, 1000]
    },
    'knn': {
        'n_neighbors': [1,5,10],
        'weights': ['uniform', 'distance']
    },
    'rfr': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10]
    },
    'grad': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.1, 0.01, 0.001],
        'max_depth': [3, 5, 7]
    },
    'svr': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10],
        'epsilon': [0.1, 0.01, 0.001]
    },
    'krr': {
        'kernel': ['linear', 'laplacian', 'rbf'],
        'alpha': [0.001, 0.01, 0.1, 1],
        'gamma': [1, 0.1, 0.01, 0.001]
    },
    'gpr': {
        'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
        'kernel': [gaussian_kernel]
    },
    'lasso': {
        'alpha': [0.001, 0.01, 0.1, 1],
        'max_iter': [1000, 2000, 3000]
    },
    'elastic': {
        'alpha': [0.001, 0.01, 0.1, 1],
        'l1_ratio': [0.2, 0.5, 0.8],
        'max_iter': [1000, 2000, 3000]
    }
}


In [5]:
def gridsearch(model,X_tr, y_tr,X_tst,y_tst):
    """
    Perform GridSearchCV for a given model

    parameters
    ----------
    model: str
        Name of model

    X_tr: np.array
        X training

    y_tr: np.array
        Y Training

    X_tst: np.array
        X test

    y_tst: np.array
        y test

    Returns
    -------
    scores: dict
        scored models
    """
    st = time.time()
    # Grab model and model parameters to perform gridsearchcv
    current_model = models[model]
    current_param_grid = param_grid[model]

    grid_search = GridSearchCV(current_model, current_param_grid, cv=5,n_jobs=-1)

    print(f'Now fitting {model}... ')

    grid_search.fit(X_tr, y_tr)

    best_model = grid_search.best_estimator_

    print(f'Completed fitting {model} in {time.time() - st:.4f} seconds. ')

    # Take the best model and evaluate using known metrics
    model=best_model
    scores = {}
    st = time.time()
    print('Now scoring model... ')
    y_tr_pred = model.predict(X_tr)
    y_tst_pred = model.predict(X_tst)

    # plt.scatter(y_tr,y_tr_pred)
    # plt.scatter(y_tst,y_tst_pred)
    # plt.scatter(y_tr, y_tr)
    # plt.scatter(y_tst,y_tst)
    # plt.show()
            
    scores['MSE_train'] = mean_squared_error(y_tr, y_tr_pred),
    scores['R2_train'] = r2_score(y_tr, y_tr_pred)
    scores['MAE_train'] = mean_absolute_error(y_tr, y_tr_pred)
    
    scores['MSE_test'] = mean_squared_error(y_tst, y_tst_pred)
    scores['R2_test'] = r2_score(y_tst, y_tst_pred)
    scores['MAE_test'] = mean_absolute_error(y_tst, y_tst_pred)
    print(f"Train R2 {scores['R2_train']:.4f}")
    print(f"Test R2 {scores['R2_test']:.4f}")
    print(f"Train MAE {scores['MAE_train']:.4f}")
    print(f"Test MAE {scores['MAE_test']:.4f}")
    print()
    return scores, best_model



In [6]:
Y=pd.read_csv(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'database','y.csv.gz'), compression='gzip',index_col=0)
Reps=['CM', 'MACCS', 'PI', 'RDKit', 'Morgan', 'SOAP']
divisions=["AB","sub"]

test_models_out={}
for r in Reps:
    test_models_out[r]={}
    for d in divisions:
        test_models_out[r][d]={}
        print(f'Start: {r}_{d}')
        X_path=os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'database',f'{r}_{d}.csv.gz')
        if os.path.exists(X_path):
            X=pd.read_csv(X_path, compression='gzip',index_col=0)
            print(X.shape[1])
            train,test=train_test_split(list(X.index), train_size=0.8,test_size=0.2,random_state=42)
            X_train, X_test, y_train, y_test = X.loc[train].to_numpy(), X.loc[test].to_numpy(), Y.loc[train].to_numpy().flatten(), Y.loc[test].to_numpy().flatten()
    
            scoring={}
            t0_init=time.perf_counter()
            for m in models.keys():
                t0=time.perf_counter()
                scores,model=gridsearch(m,X_train, y_train,X_test,y_test)
                tf=time.perf_counter()-t0
                scores['timing']=tf
                scoring[m]=scores
                print(m,tf)
            print(f"Overall {time.perf_counter()-t0_init:.2f}")
            stat_df=pd.concat([pd.DataFrame.from_dict(v).rename(index={0:k}) for k,v in scoring.items()])
            test_models_out[r][d]=stat_df
        print()
    print()

Start: CM_AB
42
Now fitting knn... 
Completed fitting knn in 0.8859 seconds. 
Now scoring model... 
Train R2 0.9879
Test R2 0.2763
Train MAE 0.6212
Test MAE 16.8484

knn 1.0121029170004476
Now fitting rfr... 
Completed fitting rfr in 43.4519 seconds. 
Now scoring model... 
Train R2 0.8903
Test R2 0.4014
Train MAE 6.3189
Test MAE 15.5350

rfr 43.583107124999515
Now fitting krr... 


/Users/grierjones/miniconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Completed fitting krr in 10.9463 seconds. 
Now scoring model... 
Train R2 0.8405
Test R2 0.3970
Train MAE 7.9196
Test MAE 15.6341

krr 11.251166959002148
Overall 55.85

Start: CM_sub
42
Now fitting knn... 
Completed fitting knn in 0.4007 seconds. 
Now scoring model... 
Train R2 1.0000
Test R2 0.3691
Train MAE 0.0000
Test MAE 15.4381

knn 0.45804766599758295
Now fitting rfr... 


/Users/grierjones/miniconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Completed fitting rfr in 45.3082 seconds. 
Now scoring model... 
Train R2 0.9221
Test R2 0.4340
Train MAE 5.2216
Test MAE 14.3435

rfr 45.42147779199877
Now fitting krr... 


...

/Users/grierjones/miniconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Completed fitting knn in 1.1043 seconds. 
Now scoring model... 
Train R2 0.6631
Test R2 0.4439
Train MAE 11.1637
Test MAE 14.4548

knn 1.6247245419981482
Now fitting rfr... 


/Users/grierjones/miniconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Completed fitting rfr in 775.9176 seconds. 
Now scoring model... 
Train R2 0.9170
Test R2 0.6015
Train MAE 5.2862
Test MAE 12.1777

rfr 776.0745715839985
Now fitting krr... 


/Users/grierjones/miniconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Process LokyProcess-42:

KeyboardInterrupt



In [ ]:
def test_shap(n_feats,model):
    """
    Test dimensionality reduction using SHapely Additive Explanations (SHAP)
    
    params
    ------
    n_feats: int
        Number of best features to reduce too
        
    model: str
        Model to test

    returns
    -------
    scores: dict
        Dictionary containing evaluation metrics

    model: trained model
    
    """
    SHAPX=X.iloc[:,sorted_cols[-n_feats:]]
    
    X_train, X_test, y_train, y_test = SHAPX.loc[train].to_numpy(), SHAPX.loc[test].to_numpy(), Y.loc[train].to_numpy(), Y.loc[test].to_numpy()
    scores,model=gridsearch(model,X_train, y_train,X_test,y_test)
    return scores,model
    

In [ ]:
def test_pca(components,model):
    """
    Test dimensionality reduction using principal component analysis (PCA)
    
    params
    ------
    components: int
        Number of dimensions to reduce too
        
    model: str
        Model to test

    returns
    -------
    scores: dict
        Dictionary containing evaluation metrics

    model: trained model
    
    """
    pca = PCA(n_components=components)
    X_train, X_test, y_train, y_test = pca.fit_transform(X.loc[train]), pca.transform(X.loc[test]), Y.loc[train].to_numpy(), Y.loc[test].to_numpy()
    scores,model=gridsearch(model,X_train, y_train,X_test,y_test)
    
    return scores,model
    

In [ ]:

# sns.barplot(data=stat_df['timing'].reset_index().melt(id_vars='index',value_vars=['timing']),x='index',y='value',hue='variable')
# sns.barplot(data=stat_df[['R2_train','R2_test']].reset_index().melt(id_vars='index',value_vars=['R2_train','R2_test']),x='index',y='value',hue='variable')


In [ ]:
model='krr'
n_feats=5
components=5

# # Normal
# scores,model=run_regular(model)

# # SHAP
# explainer = shap.Explainer(model.predict, X_test,n_jobs=-1)
# shap_values = explainer(X)
# shap.plots.bar(shap_values,max_display=16)
for i in [5,16]:
    sorted_cols=np.argsort(np.mean(np.abs(shap_values.values),axis=0))
    shap_scores,shap_model=test_shap(i,model)
    
    #PCA
    pca_scores,pca_model=test_pca(i,model)